In [1]:
import cv2, io, time, imutils, datetime
import traceback
from imutils.video import VideoStream
import numpy as np
from classes import imagenet_classes
from ovmsclient import make_grpc_client
import opencv_jupyter_ui as jcv2

from utils import crop_resize, draw_label, prep_image
from IPython.display import display, Image
import ipywidgets as widgets

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2022-08-01 11:32:23.998341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2022-08-01 11:32:23.998364: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
MODEL_NAME = "yolov5"
INPUT_NAME = "Placeholder"
RTSP_URL = "rtsp://localhost:8554/server"

In [3]:
def get_yolo_outputs(result):
    output = tf_contrib_util.make_ndarray(result.outputs['detector/yolo-v3/Conv_22/BiasAdd/YoloRegion'])
    output_numpy1 = np.array(output)
    output = tf_contrib_util.make_ndarray(result.outputs['detector/yolo-v3/Conv_14/BiasAdd/YoloRegion'])
    output_numpy2 = np.array(output)
    output = tf_contrib_util.make_ndarray(result.outputs['detector/yolo-v3/Conv_6/BiasAdd/YoloRegion'])
    output_numpy3 = np.array(output)
    
    return output_numpy1, output_numpy2, output_numpy3

In [4]:
def prep_jpeg(content):
    img = np.frombuffer(content, dtype=np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format
    img = crop_resize(img, size, size)
    img = img.astype('float32')
    
    if rgb_image:
        img = img[:, :, [2, 1, 0]]
   
    img = img.transpose(2,0,1).reshape(1,3,size,size)
    print(path, img.shape, "; data range:",np.amin(img),":",np.amax(img))
    return img

In [5]:
def run_inference(image):
    img = prep_jpeg(image)
    request = predict_pb2.PredictRequest()
    request.model_spec.name = MODEL_NAME
    request.inputs[INPUT_NAME].CopyFrom(make_tensor_proto(img, shape=(img.shape)))
    start_time = datetime.datetime.now()
    result = stub.Predict(request, 10.0) # result includes a dictionary with all model outputs
    end_time = datetime.datetime.now()
    duration = end_time - start_time
    
    return result, duration

In [6]:
rtsp_stream = VideoStream(RTSP_URL).start()
print("* RTSP Stream succesfully opened")

NameError: name 'RTSP_URL' is not defined

In [ ]:
while True:
    try: 
        frame = rtsp_stream.read()
        try: 
            if frame is not None:
                result, _ = run_inference(frame)
                output1, output2, output3 = get_yolo_outputs(result)
                jcv2.imshow("Prediction Output", frame)
            else: 
                continue 
                
            time.sleep(1/32)
        except Exception as e:
            print("error while running inference", e)
            print(traceback.format_exc())
          
    except Exception as e:
        print("error while grabbing RTSP stream", e)